# Pitch Outcome Modeling

Author: Jensen Holm <br>
April 2024

In [1]:
from constants import DATA_DIR, COMPRESSION
import polars as pl
import requests
import os

In [2]:
# get the url for our dataset of all statcast era pitches (2015-2023)
# from the huggingface API
PARQUET_URL = requests.get(
    "https://huggingface.co/api/datasets/Jensen-holm/statcast-era-pitches/parquet/default/train",
).json()[0]

print(PARQUET_URL)

https://huggingface.co/api/datasets/Jensen-holm/statcast-era-pitches/parquet/default/train/0.parquet


In [3]:
# load the dataset into a polars DataFrame
statcast_era_pitches: pl.DataFrame = pl.read_parquet(PARQUET_URL)

# print columns and their types so we can see what we're working with
print(statcast_era_pitches)

shape: (5_479_763, 92)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ pitch_typ ┆ game_date ┆ release_s ┆ release_p ┆ … ┆ of_fieldi ┆ spin_axis ┆ delta_hom ┆ delta_ru │
│ e         ┆ ---       ┆ peed      ┆ os_x      ┆   ┆ ng_alignm ┆ ---       ┆ e_win_exp ┆ n_exp    │
│ ---       ┆ str       ┆ ---       ┆ ---       ┆   ┆ ent       ┆ f32       ┆ ---       ┆ ---      │
│ str       ┆           ┆ f32       ┆ f32       ┆   ┆ ---       ┆           ┆ f32       ┆ f32      │
│           ┆           ┆           ┆           ┆   ┆ str       ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ FF        ┆ 2015-11-0 ┆ 96.099998 ┆ -2.02     ┆ … ┆ Strategic ┆ null      ┆ -0.001    ┆ -0.212   │
│           ┆ 1 00:00:0 ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ 0.0000000 ┆           ┆           ┆   ┆           ┆   

In [4]:
statcast_era_pitches.glimpse()

Rows: 5479763
Columns: 92
$ pitch_type                      <str> 'FF', 'FC', 'FF', 'FC', 'FF', 'FF', 'FF', 'FF', 'FC', 'KC'
$ game_date                       <str> '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000', '2015-11-01 00:00:00.000000000'
$ release_speed                   <f32> 96.0999984741211, 93.0999984741211, 97.0, 93.5999984741211, 97.0999984741211, 96.5, 96.5999984741211, 97.5999984741211, 92.0, 86.69999694824219
$ release_pos_x                   <f32> -2.0199999809265137, -1.659999966621399, -1.6399999856948853, -1.5800000429153442, -1.7000000476837158, -1.6200000047683716, -1.3899999856948853, -1.5099999904632568, -1.8899999856948853, -1.6200000047683716
$ release_pos_z                   <f32> 6.25, 6.239999771118164, 6.3000001

# Goal

The main goal of this project is to be able to predict how well current MLB pitchers are going to be next week, next month and next year (these will probably all be different models). In order to do this, the data will have to be structured a certain way.

In order for this to be effective and potentially useful, I want to create a model that will predict the outcome of a plate appearance with decent accuracy. Then, we can string together plate appearance predictions to predict how well a pitcher fairs against a team per 9 innings, or by a pitch amount instead maybe (whatever yeilds better results). 

It will probably be computationally expensive, but one way a team could use this to project what bullpen pitchers they want to save for which teams that they are going to face.

##### Targets(s)
- Pitcher expected run value (expected & regular woba during outing)

##### Features
(might do feature selection, or PCA to shrink this very highly dimensional dataset)
- Statcast metrics of players on teams that the pitchers team will be facing
- Split statcast metrics against hitters of similar hitting profiles
- The pitches that the pitcher throws

In [12]:
# calculate a pitchers mean woba_value for each outing, this is a metric
# that might be useful to know, especially if we can predict it well. I first want to try
# to only inlcude pitcher history agianst hitters of the same handedness of the hitter they are
# going to face (the one we are going to try to predict the woba_value on)

PITCH_MINIMUM: int = 0

pitchers_outings_df: pl.DataFrame = (
    statcast_era_pitches.group_by("game_pk", "pitcher")
    .agg(
        # Target interest: woba value
        pl.sum("woba_value").alias("total_woba_value"),
        pl.col("batter").unique().len().alias("batters_faced"),
        pl.len().alias("total_pitches"),
    )
    .filter(pl.col("total_pitches") >= PITCH_MINIMUM)
    .with_columns(
        outing_woba_value=pl.col("total_woba_value") / pl.col("batters_faced"),
    )
    .drop("total_woba_value")
)

print(pitchers_outings_df)

shape: (163_452, 5)
┌─────────┬─────────┬───────────────┬───────────────┬───────────────────┐
│ game_pk ┆ pitcher ┆ batters_faced ┆ total_pitches ┆ outing_woba_value │
│ ---     ┆ ---     ┆ ---           ┆ ---           ┆ ---               │
│ i32     ┆ i32     ┆ u32           ┆ u32           ┆ f64               │
╞═════════╪═════════╪═══════════════╪═══════════════╪═══════════════════╡
│ 491868  ┆ 521230  ┆ 3             ┆ 13            ┆ 0.0               │
│ 413919  ┆ 475857  ┆ 3             ┆ 13            ┆ 0.0               │
│ 448097  ┆ 501697  ┆ 6             ┆ 17            ┆ 0.475             │
│ 448571  ┆ 643327  ┆ 3             ┆ 15            ┆ 0.0               │
│ 661298  ┆ 592716  ┆ 9             ┆ 89            ┆ 0.877778          │
│ …       ┆ …       ┆ …             ┆ …             ┆ …                 │
│ 566000  ┆ 658792  ┆ 9             ┆ 43            ┆ 0.394444          │
│ 530130  ┆ 543883  ┆ 4             ┆ 11            ┆ 0.0               │
│ 530004  ┆ 518553

In [6]:
statcast_pitches_df: pl.DataFrame = statcast_era_pitches.join(
    other=pitchers_outings_df,
    on=("game_pk", "pitcher"),
    how="inner",
    validate="m:1",
)

print(statcast_pitches_df)

shape: (5_479_763, 95)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ pitch_typ ┆ game_date ┆ release_s ┆ release_p ┆ … ┆ delta_run ┆ batters_f ┆ total_pit ┆ outing_w │
│ e         ┆ ---       ┆ peed      ┆ os_x      ┆   ┆ _exp      ┆ aced      ┆ ches      ┆ oba_valu │
│ ---       ┆ str       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ e        │
│ str       ┆           ┆ f32       ┆ f32       ┆   ┆ f32       ┆ u32       ┆ u32       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ FF        ┆ 2015-11-0 ┆ 96.099998 ┆ -2.02     ┆ … ┆ -0.212    ┆ 4         ┆ 20        ┆ 0.225    │
│           ┆ 1 00:00:0 ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ 0.0000000 ┆           ┆           ┆   ┆           ┆   